In [4]:
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import pandas as pd
import numpy as np

In [3]:
# Set random seed.
np.random.seed(42)

# Read the processed CSV file.
df = pd.read_csv("ms_data.csv")

# Convert visit_date to datetime.
df['visit_date'] = pd.to_datetime(df['visit_date'])

#  Sort by patient_id and visit_date.
df = df.sort_values(by = ['patient_id', 'visit_date'])

# Read insurance types from `insurance.lst`.
with open('insurance.lst', 'r') as f:
    insurance_types = [line.strip() for line in f.readlines()]

# Randomly assign (but keep consistent per patient_id).
unique_patients = df['patient_id'].unique()
patient_insurance_map = {patient_id: np.random.choice(insurance_types) for patient_id in unique_patients}
df['insurance_type'] = df['patient_id'].map(patient_insurance_map)

# Generate visit costs based on insurance type. Different plans have different effects on cost.
base_costs = {'Medicare': 100,
    'Medicaid': 200,
    'Private': 50,
    'Other': 500
}

# Add random variation.
variation_factor = 0.2 # 20% variation
df['visit_cost'] = df['insurance_type'].map(base_costs) * (1 + np.random.uniform(-variation_factor, variation_factor, len(df))).round(3)

# Set appropriate data types.
df['patient_id'] = df['patient_id'].astype(str)
df['education_level'] = df['education_level'].astype(str)
df['insurance_type'] = df['insurance_type'].astype(str)

df.head()

,patient_id,visit_date,age,education_level,walking_speed,insurance_type,visit_cost
0,P0001,2020-01-23,34.28,Bachelors,4.40,Private,53.95
1,P0001,2020-04-16,34.51,Bachelors,4.19,Private,50.70
2,P0001,2020-07-03,34.72,Bachelors,4.71,Private,46.20
3,P0001,2020-10-15,35.00,Bachelors,4.86,Private,56.30
4,P0001,2020-12-29,35.21,Bachelors,4.50,Private,53.70


In [5]:
# 1. Analyze walking speed:
#    - Multiple regression with education and age (report coeffcients and confidence intervals)
#    - Account for repeated measures
#    - Test for significant trends

# Multiple regression model for walking speed with education and age as predictors
model_walk_speed = smf.mixedlm('walking_speed ~ education_level + age', df, groups=df['patient_id'])
result_walk_speed = model_walk_speed.fit()

# Coefficients and confidence intervals
coefficients_walk_speed = result_walk_speed.params
conf_int_walk_speed = result_walk_speed.conf_int()

print("Walking Speed Analysis Coefficients:")
print(coefficients_walk_speed)

print("\nWalking Speed Confidence Intervals:")
print(conf_int_walk_speed)

/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimi

Walking Speed Analysis Coefficients:
Intercept                          5.602721
education_level[T.Graduate]        0.415489
education_level[T.High School]    -0.792115
education_level[T.Some College]   -0.390001
age                               -0.030212
Group Var                          0.015059
dtype: float64

Walking Speed Confidence Intervals:
                                        0         1
Intercept                        5.582305  5.623137
education_level[T.Graduate]      0.398716  0.432262
education_level[T.High School]  -0.808972 -0.775259
education_level[T.Some College] -0.406624 -0.373379
age                             -0.030553 -0.029871
Group Var                        0.001431  0.028687


In [ ]:
# 2. Analyze costs:
#    - Simple analysis of insurance type effect
#    - Box plots and basic statistics (report coeffcients and confidence intervals)
#    - Calculate effect sizes

In [ ]:
# 3. Advanced analysis:
#    - Education age interaction effects on walking speed
#    - Control for relevant confounders
#    - Report key statistics and p-values (report coeffcients and confidence intervals)